---
# Severity Classification of Alzheimer's Disease Using MRI Images
## Jason Lu, Juan Marcucci, Dan Rossetti
---
---

# NOTEBOOK 5 - Modeling Summary and Conclusions
Notebook Summary:

This notebook provides a discussion of the modeling performed in the previous notebooks along with a summary, conclusions, and next steps.

---
---

# 1 - General Statement on Model Stability and Reproducibility:
Despite every effort being made to set random seeds at the model-level and at the notebook-level in the imports, the models are incapable of producing consistent results.  With that, the observations and conclusions are written based on the most recent execution of each model.  This variability was not discovered until very late in the modeling process, preventing an effective remedy from being implemented.

Variability:  
Some models have been observed to produce accuracy results varying from the low 50% to as high as the low 70% range from run-to-run

Possible Causes:
* The ‘image_dataset_from_directory’ function is incapable of producing consistent results when Shuffle is set to ‘True’ even with ‘seed’ set to a constant value.  It is believed that ‘shuffle’ must be set to ‘true’ in order to sufficiently randomize the images that are populated into the training and validation datasets, otherwise, pulling the images in the order of the files in the directory may result in a validation dataset that is entirely one class. It appears that each time this function is called, a totally new set of images is selected for the training and validation sets.  This can be observed by unpacking the keras object created by the function with a for loop and observing the differences in the classes of the first batch of target variables.  Running the code shown below will demonstrate this effect.
* For some models, the validation accuracy scores at each epoch can vary drastically, soaring and plummeting over the course of even a few epochs
* Lastly, there is some variability introduced by the fact that random GPUs from Google Colab are being used to train the neural networks
```
# Given:
train_ds = image_dataset_from_directory(
    '/content/ga_project_4/data/Alzheimer_s Dataset/train',
    image_size = (208, 176),
    batch_size = 32,
    seed = 42,
    validation_split = 0.20,
    subset = 'training',
    label_mode = 'categorical',
)
```
```
# Unpack train_ds with:
test1 = [y for x, y in train_ds][0]
test2 = [y for x, y in train_ds][0]
```
```
# Test equality with:
test1==test2
```
```
output:
<tf.Tensor: shape=(32, 4), dtype=bool, numpy=
array([[ True,  True,  True,  True],
       [False,  True, False,  True],
       [False,  True, False,  True],
       [ True,  True, False, False],
       [ True,  True, False, False],
       [ True,  True,  True,  True],
       [ True,  True, False, False],
       [ True,  True, False, False],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True, False, False],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True, False, False],
       [ True,  True, False, False],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True, False, False],
       [ True,  True, False, False],
       [ True,  True, False, False],
       [False,  True,  True, False],
       [ True,  True, False, False],
       [ True,  True,  True,  True],
       [False,  True,  True, False],
       [False,  True,  True, False],
       [ True,  True,  True,  True],
       [ True,  True, False, False],
       [ True,  True,  True,  True],
       [ True,  True, False, False]])>
```

Above, the equalities of the samples in the first batch of image classes (which were one hot encoded by 'image_dataset_from_directory') are shown.  It can be seen that desipite 'test1' and 'test2' being set to the exact same list comrehension, not all values are in the arrays yielded are the same.  This would suggest that the values output by this function are not consistent even from run-to-run.


# 2 - Modeling Results

## 2.1 - Multi-class Classification Models
### 2.1.1 - Manually Built Models

| CLASS              | Initial | Optimized | Optimized with Class Weights | Optimized with Image Aug. | Optimized With Regularization |
|--------------------|---------|-----------|------------------------------|---------------------------|-------------------------------|
| Model Accuracy     | 0.573   | 0.527     | 0.672                        | 0.522                     | 0.522                         |
| Recall No Dementia | 0.47    | 0.65      | 0.82                         | 0.92                      | 0.92                          |
| Recall Very Mild   | 0.83    | 0.29      | 0.58                         | 0.17                      | 0.17                          |
| Recall Mild        | 0.34    | 0.64      | 0.41                         | 0.03                      | 0.03                          |
| Recall Moderate    | 0.17    | 0.83      | 0.33                         | 0.00                      | 0.00                          |

### 2.1.2 - Pre-Trained Models

| CLASS              | ENetB0 | VGG19 | VGG16 | IncepV3 | Optimized With Regularization |
|--------------------|--------|-------|-------|---------|-------------------------------|
| Model Accuracy     | 0.553  | 0.549 | 0.397 | 0.714   | 0.522                         |
| Recall No Dementia | 0.54   | 0.55  | 0.07  | 0.75    | 0.92                          |
| Recall Very Mild   | 0.69   | 0.43  | 0.96  | 0.77    | 0.17                          |
| Recall Mild        | 0.27   | 0.84  | 0.18  | 0.48    | 0.03                          |
| Recall Moderate    | 0.33   | 0.67  | 0.00  | 0.08    | 0.00                          |


## 2.2 - Binary Classification Models

| CLASS              | Initial | Optimized | IncepV3 |
|--------------------|---------|-----------|---------|
| Model Accuracy     | 0.640   | 0.812     | 0.588   |
| Recall No Dementia | 0.50    | 0.83      | 1.00    |
| Recall Very Mild   | 0.54    | 0.78      | 0.00    |

# 3 - Summary
* MRI brain scan images were imported from Kaggle and contained training and test datasets with images of brains with no dementia, very mild dementia, mild dementia and moderate dementia
* These images were used to train a variety of manually built and pre-trained models
* After these models were used to perform multi-class image classification, they were adapted to become binary classification models to classify images in only the no dementia and very mild dementia classes
* Model performances ranged between the low 50% to low 70% range for the multi-class models and low 50% to low 80% for the binary classification models
The best multi-class model achieved and accuracy of around 71% and the best binary classification model achieved an accuracy of approximately 81%

# 4 - Conclusions
* Models can be built that will perform better than the null model, but are not yet to the level of accuracy or recall required for use in a diagnostic situation
* Binary classification models appear to perform better at classifying the no dementia and very mild dementia classes than the multiclass models. While more modeling is needed, a binary model may be preferable for early diagnostic purposes

# 5 - Next Steps
Additional modeling is required to improve model performances. The following steps should be taken in subsequent modeling stages:
* Measures should be taken to enhance model stability and the above models should then be further verified
* Pretrained models are available which are specifically geared towards medical imaging applications and should be implemented in the next stages of modeling
Additional image augmentation techniques could be explored
* A deeper exploration of image classification techniques could be performed by referencing models which have more successfully classified these images